In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

**Connection String**

In [2]:
import sqlalchemy
# Connect to database
PWD='root'
USR='root'
SQLALCHEMY_DATABASE_URI = 'mysql+pymysql://{}:{}@localhost:3306/moviesdb'.format(USR, PWD)
engine = sqlalchemy.create_engine(SQLALCHEMY_DATABASE_URI)


**Basic**

In [3]:
df = pd.read_sql("movies",engine)
print(df.head())
print(df.columns)

   movie_id                                        title   industry  \
0       101                             K.G.F: Chapter 2  Bollywood   
1       102  Doctor Strange in the Multiverse of Madness  Hollywood   
2       103                        Thor: The Dark World   Hollywood   
3       104                              Thor: Ragnarok   Hollywood   
4       105                      Thor: Love and Thunder   Hollywood   

   release_year  imdb_rating          studio  language_id  
0          2022          8.4   Hombale Films            3  
1          2022          7.0  Marvel Studios            5  
2          2013          6.8  Marvel Studios            5  
3          2017          7.9  Marvel Studios            5  
4          2022          6.8  Marvel Studios            5  
Index(['movie_id', 'title', 'industry', 'release_year', 'imdb_rating',
       'studio', 'language_id'],
      dtype='object')


In [4]:

query = """
select *,
	case when unit ="Billions" then revenue*1000
         when unit = "Thousands" then revenue/1000
		 else revenue
    end as revenue_M,
    if(currency = "USD",77,1) as converstion
 from moviesdb.financials
"""
df_fin = pd.read_sql(query,engine)
print(df_fin.head())

   movie_id  budget  revenue      unit currency  revenue_M  converstion
0       101     1.0     12.5  Billions      INR    12500.0            1
1       102   200.0    954.8  Millions      USD      954.8           77
2       103   165.0    644.8  Millions      USD      644.8           77
3       104   180.0    854.0  Millions      USD      854.0           77
4       105   250.0    670.0  Millions      USD      670.0           77


In [5]:
df_fin['revenue_M_Inr'] = df_fin['revenue_M']*df_fin['converstion']
print(df_fin.head())

   movie_id  budget  revenue      unit currency  revenue_M  converstion  \
0       101     1.0     12.5  Billions      INR    12500.0            1   
1       102   200.0    954.8  Millions      USD      954.8           77   
2       103   165.0    644.8  Millions      USD      644.8           77   
3       104   180.0    854.0  Millions      USD      854.0           77   
4       105   250.0    670.0  Millions      USD      670.0           77   

   revenue_M_Inr  
0        12500.0  
1        73519.6  
2        49649.6  
3        65758.0  
4        51590.0  


**Join**

In [6]:
query = """
select m.movie_id,m.title,f.budget,f.revenue,f.currency,f.unit
from moviesdb.movies m
join moviesdb.financials f on m.movie_id = f.movie_id
"""
df_movie_fin = pd.read_sql(query,engine)
df_movie_fin.shape

(37, 6)

In [7]:
query = """
select m.movie_id,m.title,f.budget,f.revenue,f.currency,f.unit
from moviesdb.movies m
left join moviesdb.financials f on m.movie_id = f.movie_id
union 
select f.movie_id,m.title,f.budget,f.revenue,f.currency,f.unit
from moviesdb.movies m
right join moviesdb.financials f on m.movie_id = f.movie_id
"""
df_movie_fin = pd.read_sql(query,engine)
df_movie_fin.shape

(42, 6)

In [8]:
query = """
select m.movie_id,m.title,f.budget,f.revenue,f.currency,f.unit
from moviesdb.movies m
left join moviesdb.financials f on m.movie_id = f.movie_id
"""
df_movie_fin_left = pd.read_sql(query,engine)
df_movie_fin_left.shape

(39, 6)

In [9]:
query = """
select f.movie_id,m.title,f.budget,f.revenue,f.currency,f.unit
from moviesdb.movies m
right join moviesdb.financials f on m.movie_id = f.movie_id
"""
df_movie_fin_right = pd.read_sql(query,engine)
df_movie_fin_right.shape

(40, 6)

In [10]:
df_movie_concat = pd.concat([df_movie_fin_left,df_movie_fin_right],axis=1)
df_movie_concat.shape

(40, 12)

In [11]:
df_movie_merge = pd.merge(df_movie_fin_left,df_movie_fin_right,on='movie_id')
print(df_movie_merge.shape)

(37, 11)


**Complex Queries**

subquery

In [12]:
# return single value in where condition
query = """
select * from moviesdb.movies 
where imdb_rating = (select max(imdb_rating) from moviesdb.movies)
"""
df_movie_fin_right = pd.read_sql(query,engine)
df_movie_fin_right.head()

,movie_id,title,industry,release_year,imdb_rating,studio,language_id
0,111,The Shawshank Redemption,Hollywood,1994,9.3,Castle Rock Entertainment,5


In [13]:
# return multipule value in where condition
query = """
select * from moviesdb.movies 
where imdb_rating In (
(select max(imdb_rating) from moviesdb.movies) , 
(select min(imdb_rating) from moviesdb.movies)
);
"""
df_movie_fin_right = pd.read_sql(query,engine)
df_movie_fin_right.head()

,movie_id,title,industry,release_year,imdb_rating,studio,language_id
0,111,The Shawshank Redemption,Hollywood,1994,9.3,Castle Rock Entertainment,5
1,139,Race 3,Bollywood,2018,1.9,Salman Khan Films,1


In [14]:
# return multipule value in where condition
query = """
select *,
Year(current_date())-birth_year as age
 from moviesdb.actors
"""
df_actors = pd.read_sql(query,engine)
df_actors.head()

,actor_id,name,birth_year,age
0,50,Yash,1986,37
1,51,Sanjay Dutt,1959,64
2,52,Benedict Cumberbatch,1976,47
3,53,Elizabeth Olsen,1989,34
4,54,Chris Hemsworth,1983,40


co-related query

In [15]:
# join
query = """
select ma.actor_id,a.name,count(ma.movie_id) as movie_count
from moviesdb.movie_actor as ma
left join moviesdb.actors a on a.actor_id = ma.actor_id
group by ma.actor_id,a.name 
order by movie_count desc
"""
df_actors = pd.read_sql(query,engine)
df_actors.head()

,actor_id,name,movie_count
0,54,Chris Hemsworth,5
1,95,Chris Evans,4
2,61,Aamir Khan,3
3,51,Sanjay Dutt,2
4,59,Shah Rukh Khan,2


In [16]:
#co-related query
query = """
select m.movie_id,m.title,
(
select count(*) from movie_actor where movie_id = m.movie_id
) as actors_count
from moviesdb.movies m
order by actors_count desc
"""
df_actors = pd.read_sql(query,engine)
df_actors.head()

,movie_id,title,actors_count
0,108,3 Idiots,3
1,109,Kabhi Khushi Kabhie Gham,3
2,103,Thor: The Dark World,3
3,125,Avengers: Endgame,3
4,126,Avengers: Infinity War,3


CTE

In [17]:
#co-related query
query = """
with cte1 as (
SELECT *,
(revenue - budget) as pct,
if(currency = 'USD',revenue*77,revenue) as revenue_inr
FROM moviesdb.financials
)select *,
(CASE
WHEN unit = 'Billions' then revenue_inr * 1000
when unit = 'Thousands' then revenue_inr/1000
else revenue_inr end) as revenue_mln
from cte1;
"""
df_actors = pd.read_sql(query,engine)
df_actors.head()

,movie_id,budget,revenue,unit,currency,pct,revenue_inr,revenue_mln
0,101,1.0,12.5,Billions,INR,11.5,12.5,12500.0
1,102,200.0,954.8,Millions,USD,754.8,73519.6,73519.6
2,103,165.0,644.8,Millions,USD,479.8,49649.6,49649.6
3,104,180.0,854.0,Millions,USD,674.0,65758.0,65758.0
4,105,250.0,670.0,Millions,USD,420.0,51590.0,51590.0
